In [ ]:
import torch
import numpy as np
from model import Model
from ploter import Ploter

In [ ]:
# Body
n_samples = 1000
# 2min15 pour 1000 / 100
n_points = 100 # points per sample

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 1
stop_after_epochs = 3
model = Model(device, seed, stop_after_epochs)

raw_low = model.to_tensor([3])
raw_high = model.to_tensor([5])
model.set_prior(raw_low, raw_high)
stride = 1
pre_N = 200
preruns = 1
model.set_simulator(stride, pre_N, preruns)

raw_data, raw_parameters = model.simulate_raw_data(n_samples, n_points)
model.set_normalizer(raw_data, raw_parameters)
data = model.normalizer.normalize_data(raw_data)
parameters = model.normalizer.normalize_parameters(raw_parameters)

model.build_default()

In [ ]:
model.train(data, parameters) # pour l'instant une dimension de point (il faut faire encoder)
Ploter.plot_loss(model.neural_network)

In [ ]:
# Tests
print("Raw data")
print(raw_data.shape)
print(raw_data[0,:6])

print("\nRaw parameters")
print(raw_parameters.shape)
print(raw_parameters[:6])

print("\nNormalized data")
print(data.shape)
print(data[0,:6])
print(data.mean(dim=(0,1)))
print(data.std(dim=(0,1)))

print("\nNormalized parameters")
print(parameters.shape)
print(parameters[:6])
print(parameters.mean(dim=0))
print(parameters.std(dim=0))

In [ ]:
true_parameter, observed_data = model.get_random_true_parameter(n_points)
print("true parameter", true_parameter)
print("observed data", observed_data[:6])
n_parameters = 100
sampled_parameters = model.draw_parameters_from_predicted_posterior(observed_data, n_parameters)
print("sampled parameters mean", sampled_parameters.mean())


In [ ]:
Ploter.plot_a_sample(model, data[0], parameters[0])

In [ ]:
true_parameter, observed_sample = model.get_random_true_parameter(n_points)
print("True parameter", true_parameter)
print("Observed data", observed_sample.mean(dim=(0)), observed_sample.std(dim=0))
n_sampled_parameters = 1000
sampled_parameters = model.draw_parameters_from_predicted_posterior(observed_sample, n_sampled_parameters)
print("Predicted parameter mean", sampled_parameters.mean())
Ploter.plot_a_posterior(model, sampled_parameters)

In [ ]:
n_similar = 5
Ploter.plot_similar_data(model, observed_sample, n_similar, n_points)